In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

def human_delay():
    time.sleep(random.uniform(1, 3)) 

# Setting up the Chrome WebDriver
options = webdriver.ChromeOptions()
options.headless = True 
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36") # Set headless mode to True to run without opening a browser window
driver = webdriver.Chrome(options=options)

data_inc = {'car_name': [], 'car_info': [], 'car_year_hand': [], 'car_price': [],'car_tag': []}
for i in range(1, 201):
    url = f"https://www.yad2.co.il/vehicles/cars?page={i}"
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))  # Simple condition
    except TimeoutException:
        print("Timed out waiting for page to load: ", url)
        continue
    
    # Scroll to the bottom of the page (if needed)
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    human_delay()  # Allow extra time for any lazy-loaded elements
    
    # Random scroll up and down
    for _ in range(random.randint(1, 3)):
        scroll_height = random.randint(100, 500)
        driver.execute_script(f"window.scrollBy(0, -{scroll_height});")
        human_delay()
        driver.execute_script(f"window.scrollBy(0, {scroll_height});")
        human_delay()
        
    # Extract HTML content
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    car_boxes = soup.find_all('div', class_='feed-item-base_feedItemBox__5WVY1 scale-up-on-hover_scaleUpOnHover__QSVDw')
    for box in car_boxes:
        car_name = box.find('span', class_='feed-item-info_heading__k5pVC').get_text(strip=True)
        car_info = box.find('span', class_='feed-item-info_marketingText__eNE4R').get_text(strip=True)
        car_year_hand = box.find('span', class_='feed-item-info_yearAndHandBox___JLbc').get_text(strip=True)
        car_price = box.find('span', class_='price_price__xQt90').get_text(strip=True)
        car_tags = box.find('div', class_='tag-list_upgradeItemFlagBox__qPadL')
        if car_tags:
            car_tags_texts = [tag.get_text(strip=True) for tag in car_tags.find_all('span', class_='tag-list_tag__EX6gE flag_baseFlag__PIHA5 flag_grayFlag__1s_oj')]
        else:
            car_tags_texts = ['']
        # data_inc['car_tag'].extend(car_tags_texts)
        data_inc['car_name'].append(car_name)
        data_inc['car_info'].append(car_info)
        data_inc['car_year_hand'].append(car_year_hand)
        data_inc['car_price'].append(car_price)
        data_inc['car_tag'].append(','.join(car_tags_texts))
        # if title_element and date_element:
        #     title = title_element.get_text(strip=True)
        #     date = date_element.get_text(strip=True)
        #     link = title_element.find('a')['href'] if title_element.find('a') else None
        #     data_inc['title'].append(title)
        #     data_inc['date'].append(date)
        #     data_inc['link'].append(link)
# Close the driver after the loop
driver.quit()# Display the first few rows of the DataFrame


In [8]:
df_car_prices = pd.DataFrame(data_inc)
df_car_prices

,car_name,car_info,car_year_hand,car_price,car_tag
0,ניסאן קשקאי,Acenta אוט' 1.2 (115 כ''ס),2015 • יד 3,"43,375 ₪","חסכוני,גלגלי מגנזיום,טסט בתוקף"
1,לקסוס RX350,4X4 Premium אוט' 3.5 (276 כ''ס),2007 • יד 5,"25,000 ₪","גלגלי מגנזיום,מנוע חזק,טסט בתוקף"
2,שברולט סוואנה ארוך,Max סגור אוט' דיזל 4.5 טון 2.8 (181 כ''ס),2020 • יד 1,"147,000 ₪","מנוע חזק,טסט בתוקף,יד ראשונה"
3,שברולט ספארק,"LT אוט' 1.4 (98 כ""ס)",2017 • יד 2,"3,000 ₪","בקרת שמירת מרחק,תא מטען גדול,טסט בתוקף"
4,מרצדס EQV,"Avantgarde חשמלי 4X2 (204 כ""ס)",2023 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,גלגלי מגנזיום,חשמלי"
...,...,...,...,...,...
7955,ב.מ.וו סדרה 1,118I S אוט' 5 דל' 1.5 (136 כ''ס),2018 • יד 2,לא צוין מחיר,"בקרת שמירת מרחק,בהזדמנות"
7956,ניסאן ג'וק / Juke,"Acenta אוט' 1.6 (117 כ""ס)",2016 • יד 3,"37,990 ₪","גלגלי מגנזיום,טסט בתוקף"
7957,סיטרואן ברלינגו דור 3 (2018 ואילך),Shine ידני דיזל 5 מק' 1.5 (100 כ''ס),2021 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,חסכוני"
7958,אאודי Q5,4X4 Off Road אוט' 2.0 (225 כ''ס),2014 • יד 2,"69,900 ₪","גלגלי מגנזיום,טסט בתוקף"


In [9]:
df_car_prices.to_csv('yad2_v3_04_06_24_v3.csv', index=False)




In [10]:
# Read the CSV files
df_v1 = pd.read_csv('yad2_v3_04_06_24_v1.csv')
df_v2 = pd.read_csv('yad2_v3_04_06_24_v2.csv')
df_v3 = pd.read_csv('yad2_v3_04_06_24_v3.csv')

# Combine the datasets
combined_df = pd.concat([df_v1, df_v2, df_v3], ignore_index=True)

# Display the first few rows of the combined DataFrame
combined_df.head()


,car_name,car_info,car_year_hand,car_price,car_tag
0,אלפא רומיאו מיטו / MITO,"QV Volante אוט' 1.4 (170 כ""ס)",2016 • יד 2,"48,000 ₪","חסכוני,גלגלי מגנזיום,טסט בתוקף"
1,מרצדס E-Class,4X4 E300E AMG Line הייבריד אוט' 2.0 (211 כ''ס),2021 • יד 1,"329,000 ₪","גלגלי מגנזיום,היברידי,טסט בתוקף"
2,סאנגיונג רקסטון,4X4 RX270 אוט' דיזל 7 מק' 2.7 (165 כ''ס),2012 • יד 3,"52,000 ₪","מיכל דלק גדול,גלגלי מגנזיום,טסט בתוקף"
3,יונדאי סנטה פה,Luxury הייבריד אוט' 1.6 (226 כ''ס),2022 • יד 2,"215,000 ₪","בקרת שיוט אדפטיבית,גלגלי מגנזיום,היברידי"
4,סקודה סופרב (2019 ואילך),Ambition אוט' 2.0 (190 כ''ס),2021 • יד 1,"132,000 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום"


In [11]:
# Count duplicates in the combined DataFrame
duplicate_count = combined_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")


Number of duplicate rows: 1976


In [12]:
# Drop duplicate rows from the combined DataFrame
combined_df = combined_df.drop_duplicates()

# Display the first few rows of the DataFrame after dropping duplicates
combined_df


,car_name,car_info,car_year_hand,car_price,car_tag
0,אלפא רומיאו מיטו / MITO,"QV Volante אוט' 1.4 (170 כ""ס)",2016 • יד 2,"48,000 ₪","חסכוני,גלגלי מגנזיום,טסט בתוקף"
1,מרצדס E-Class,4X4 E300E AMG Line הייבריד אוט' 2.0 (211 כ''ס),2021 • יד 1,"329,000 ₪","גלגלי מגנזיום,היברידי,טסט בתוקף"
2,סאנגיונג רקסטון,4X4 RX270 אוט' דיזל 7 מק' 2.7 (165 כ''ס),2012 • יד 3,"52,000 ₪","מיכל דלק גדול,גלגלי מגנזיום,טסט בתוקף"
3,יונדאי סנטה פה,Luxury הייבריד אוט' 1.6 (226 כ''ס),2022 • יד 2,"215,000 ₪","בקרת שיוט אדפטיבית,גלגלי מגנזיום,היברידי"
4,סקודה סופרב (2019 ואילך),Ambition אוט' 2.0 (190 כ''ס),2021 • יד 1,"132,000 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום"
...,...,...,...,...,...
8955,ב.מ.וו סדרה 1,118I S אוט' 5 דל' 1.5 (136 כ''ס),2018 • יד 2,לא צוין מחיר,"בקרת שמירת מרחק,בהזדמנות"
8956,ניסאן ג'וק / Juke,"Acenta אוט' 1.6 (117 כ""ס)",2016 • יד 3,"37,990 ₪","גלגלי מגנזיום,טסט בתוקף"
8957,סיטרואן ברלינגו דור 3 (2018 ואילך),Shine ידני דיזל 5 מק' 1.5 (100 כ''ס),2021 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,חסכוני"
8958,אאודי Q5,4X4 Off Road אוט' 2.0 (225 כ''ס),2014 • יד 2,"69,900 ₪","גלגלי מגנזיום,טסט בתוקף"


In [14]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='en')

# Function to translate text using deep_translator
def deep_translate_text(text, target='en'):
    return translator.translate(text)

# Apply the translation function to the DataFrame
combined_df['car_name_en'] = combined_df['car_name'].apply(deep_translate_text)

combined_df

,car_name,car_info,car_year_hand,car_price,car_tag,car_name_en
0,אלפא רומיאו מיטו / MITO,"QV Volante אוט' 1.4 (170 כ""ס)",2016 • יד 2,"48,000 ₪","חסכוני,גלגלי מגנזיום,טסט בתוקף",Alfa Romeo Mito / MITO
1,מרצדס E-Class,4X4 E300E AMG Line הייבריד אוט' 2.0 (211 כ''ס),2021 • יד 1,"329,000 ₪","גלגלי מגנזיום,היברידי,טסט בתוקף",Mercedes E-Class
2,סאנגיונג רקסטון,4X4 RX270 אוט' דיזל 7 מק' 2.7 (165 כ''ס),2012 • יד 3,"52,000 ₪","מיכל דלק גדול,גלגלי מגנזיום,טסט בתוקף",Ssangyong Rexton
3,יונדאי סנטה פה,Luxury הייבריד אוט' 1.6 (226 כ''ס),2022 • יד 2,"215,000 ₪","בקרת שיוט אדפטיבית,גלגלי מגנזיום,היברידי",Hyundai Santa Fe
4,סקודה סופרב (2019 ואילך),Ambition אוט' 2.0 (190 כ''ס),2021 • יד 1,"132,000 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום",Skoda Superb (2019 onwards)
...,...,...,...,...,...,...
8955,ב.מ.וו סדרה 1,118I S אוט' 5 דל' 1.5 (136 כ''ס),2018 • יד 2,לא צוין מחיר,"בקרת שמירת מרחק,בהזדמנות",BMW 1 Series
8956,ניסאן ג'וק / Juke,"Acenta אוט' 1.6 (117 כ""ס)",2016 • יד 3,"37,990 ₪","גלגלי מגנזיום,טסט בתוקף",Nissan Juke / Juke
8957,סיטרואן ברלינגו דור 3 (2018 ואילך),Shine ידני דיזל 5 מק' 1.5 (100 כ''ס),2021 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,חסכוני",Citroen Berlingo 3rd generation (2018 onwards)
8958,אאודי Q5,4X4 Off Road אוט' 2.0 (225 כ''ס),2014 • יד 2,"69,900 ₪","גלגלי מגנזיום,טסט בתוקף",Audi Q5


In [15]:
combined_df['car_info_en'] = combined_df['car_info'].apply(deep_translate_text)

In [16]:
combined_df['car_tags_en'] = combined_df['car_tag'].apply(lambda x: deep_translate_text(x) if pd.notna(x) else x)

In [17]:
combined_df.to_csv('yad2_v3_04_06_24_final.csv', index=False)


In [18]:
combined_df

,car_name,car_info,car_year_hand,car_price,car_tag,car_name_en,car_info_en,car_tags_en
0,אלפא רומיאו מיטו / MITO,"QV Volante אוט' 1.4 (170 כ""ס)",2016 • יד 2,"48,000 ₪","חסכוני,גלגלי מגנזיום,טסט בתוקף",Alfa Romeo Mito / MITO,QV Volante auto 1.4 (170 hp),"Economical, magnesium wheels, fully tested"
1,מרצדס E-Class,4X4 E300E AMG Line הייבריד אוט' 2.0 (211 כ''ס),2021 • יד 1,"329,000 ₪","גלגלי מגנזיום,היברידי,טסט בתוקף",Mercedes E-Class,4X4 E300E AMG Line hybrid auto 2.0 (211 hp),"Magnesium wheels, hybrid, validly tested"
2,סאנגיונג רקסטון,4X4 RX270 אוט' דיזל 7 מק' 2.7 (165 כ''ס),2012 • יד 3,"52,000 ₪","מיכל דלק גדול,גלגלי מגנזיום,טסט בתוקף",Ssangyong Rexton,4X4 RX270 OT diesel 7 mk 2.7 (165 hp),"Large fuel tank, magnesium wheels, fully tested"
3,יונדאי סנטה פה,Luxury הייבריד אוט' 1.6 (226 כ''ס),2022 • יד 2,"215,000 ₪","בקרת שיוט אדפטיבית,גלגלי מגנזיום,היברידי",Hyundai Santa Fe,Luxury Hybrid Auto 1.6 (226 HP),"Adaptive cruise control, magnesium wheels, hybrid"
4,סקודה סופרב (2019 ואילך),Ambition אוט' 2.0 (190 כ''ס),2021 • יד 1,"132,000 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק,גלגלי מגנזיום",Skoda Superb (2019 onwards),Ambition auto 2.0 (190 hp),"Adaptive cruise control, cruise control, magne..."
...,...,...,...,...,...,...,...,...
8955,ב.מ.וו סדרה 1,118I S אוט' 5 דל' 1.5 (136 כ''ס),2018 • יד 2,לא צוין מחיר,"בקרת שמירת מרחק,בהזדמנות",BMW 1 Series,118I S Ot' 5 Del. 1.5 (136 HP),"Distance control, on occasion"
8956,ניסאן ג'וק / Juke,"Acenta אוט' 1.6 (117 כ""ס)",2016 • יד 3,"37,990 ₪","גלגלי מגנזיום,טסט בתוקף",Nissan Juke / Juke,Acenta auto 1.6 (117 hp),"Magnesium wheels, fully tested"
8957,סיטרואן ברלינגו דור 3 (2018 ואילך),Shine ידני דיזל 5 מק' 1.5 (100 כ''ס),2021 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,חסכוני",Citroen Berlingo 3rd generation (2018 onwards),Shine manual diesel 5 mk 1.5 (100 hp),"Distance control, economical"
8958,אאודי Q5,4X4 Off Road אוט' 2.0 (225 כ''ס),2014 • יד 2,"69,900 ₪","גלגלי מגנזיום,טסט בתוקף",Audi Q5,4X4 Off Road Auto 2.0 (225 HP),"Magnesium wheels, fully tested"


In [20]:
df_first = pd.read_csv('yad2_cars_data_final.csv')
df_second = pd.read_csv('yad2_cars_data_more2krows.csv')
df_third = pd.read_csv('yad2_v3_04_06_24_final.csv')

df_final = pd.concat([df_first, df_second, df_third], ignore_index=True)
df_final


,car_name,car_info,car_year_hand,car_price,car_tag,car_name_en,car_info_en,car_tags_en
0,ג'יפ / Jeep רנגלר ארוך,4X4 Unlimited Sahara אוט' 5 דל' 2.0 (272 כ''ס),2021 • יד 1,"300,000 ₪","גלגלי מגנזיום,טסט בתוקף,יד ראשונה",Jeep / Jeep Wrangler long,4X4 Unlimited Sahara Ot' 5 Del' 2.0 (272 HP),"Magnesium wheels, fully tested, first hand"
1,אאודי Q7,4X4 Premium חשמלי דיזל אוט' 5 מק' 3.0 (374 כ''ס),2018 • יד 2,"216,000 ₪","חסכוני,היברידי,טסט בתוקף",Audi Q7,4X4 Premium electric diesel auto 5 mk 3.0 (374...,"Economical, hybrid, valid test"
2,סובארו פורסטר,4X4 B אוט' 2.0 (150 כ''ס) [2008-2011],2011 • יד 3,"42,000 ₪","שטח קשוח,טסט בתוקף,מערכת מולטימדיה",Subaru Forester,4X4 B auto 2.0 (150 hp) [2008-2011],"Hard terrain, valid test, multimedia system"
3,ג'יפ / Jeep גרנד צ'ירוקי,4X4 Laredo אוט' 5 מק' 3.6 (286 כ''ס),2019 • יד 1,"160,000 ₪","בקרת שמירת מרחק,גלגלי מגנזיום,טסט בתוקף",Jeep / Jeep Grand Cherokee,4X4 Laredo auto 5 mk 3.6 (286 hp),"Distance control, magnesium wheels, valid test"
4,טויוטה לנד קרוזר ארוך,"4X4 STD אוט' דיזל 5 מק' 3.0 (173 כ""ס)",2008 • יד 1,"123,000 ₪","מיכל דלק גדול,טסט בתוקף,יד ראשונה",Toyota Land Cruiser long,4X4 STD OT diesel 5 mk 3.0 (173 hp),"Large fuel tank, fully tested, first hand"
...,...,...,...,...,...,...,...,...
19727,ב.מ.וו סדרה 1,118I S אוט' 5 דל' 1.5 (136 כ''ס),2018 • יד 2,לא צוין מחיר,"בקרת שמירת מרחק,בהזדמנות",BMW 1 Series,118I S Ot' 5 Del. 1.5 (136 HP),"Distance control, on occasion"
19728,ניסאן ג'וק / Juke,"Acenta אוט' 1.6 (117 כ""ס)",2016 • יד 3,"37,990 ₪","גלגלי מגנזיום,טסט בתוקף",Nissan Juke / Juke,Acenta auto 1.6 (117 hp),"Magnesium wheels, fully tested"
19729,סיטרואן ברלינגו דור 3 (2018 ואילך),Shine ידני דיזל 5 מק' 1.5 (100 כ''ס),2021 • יד 1,לא צוין מחיר,"בקרת שמירת מרחק,חסכוני",Citroen Berlingo 3rd generation (2018 onwards),Shine manual diesel 5 mk 1.5 (100 hp),"Distance control, economical"
19730,אאודי Q5,4X4 Off Road אוט' 2.0 (225 כ''ס),2014 • יד 2,"69,900 ₪","גלגלי מגנזיום,טסט בתוקף",Audi Q5,4X4 Off Road Auto 2.0 (225 HP),"Magnesium wheels, fully tested"


In [21]:
df_final = df_final.drop_duplicates()


In [23]:
df_final.to_csv('yad2_13k_rows_final_05_06_24.csv', index=False)


In [24]:
df_final

,car_name,car_info,car_year_hand,car_price,car_tag,car_name_en,car_info_en,car_tags_en
0,ג'יפ / Jeep רנגלר ארוך,4X4 Unlimited Sahara אוט' 5 דל' 2.0 (272 כ''ס),2021 • יד 1,"300,000 ₪","גלגלי מגנזיום,טסט בתוקף,יד ראשונה",Jeep / Jeep Wrangler long,4X4 Unlimited Sahara Ot' 5 Del' 2.0 (272 HP),"Magnesium wheels, fully tested, first hand"
1,אאודי Q7,4X4 Premium חשמלי דיזל אוט' 5 מק' 3.0 (374 כ''ס),2018 • יד 2,"216,000 ₪","חסכוני,היברידי,טסט בתוקף",Audi Q7,4X4 Premium electric diesel auto 5 mk 3.0 (374...,"Economical, hybrid, valid test"
2,סובארו פורסטר,4X4 B אוט' 2.0 (150 כ''ס) [2008-2011],2011 • יד 3,"42,000 ₪","שטח קשוח,טסט בתוקף,מערכת מולטימדיה",Subaru Forester,4X4 B auto 2.0 (150 hp) [2008-2011],"Hard terrain, valid test, multimedia system"
3,ג'יפ / Jeep גרנד צ'ירוקי,4X4 Laredo אוט' 5 מק' 3.6 (286 כ''ס),2019 • יד 1,"160,000 ₪","בקרת שמירת מרחק,גלגלי מגנזיום,טסט בתוקף",Jeep / Jeep Grand Cherokee,4X4 Laredo auto 5 mk 3.6 (286 hp),"Distance control, magnesium wheels, valid test"
4,טויוטה לנד קרוזר ארוך,"4X4 STD אוט' דיזל 5 מק' 3.0 (173 כ""ס)",2008 • יד 1,"123,000 ₪","מיכל דלק גדול,טסט בתוקף,יד ראשונה",Toyota Land Cruiser long,4X4 STD OT diesel 5 mk 3.0 (173 hp),"Large fuel tank, fully tested, first hand"
...,...,...,...,...,...,...,...,...
19722,טויוטה אייגו (AYGO),Plus רובוטית 1.0 (69 כ''ס),2013 • יד 4,"19,900 ₪","חסכוני,טסט בתוקף",Toyota Aygo (AYGO),Plus robotic 1.0 (69 hp),"Economical, valid test"
19724,רנו מגאן גראנד קופה,"Intense אוט' דיזל 1.5 (110 כ""ס)",2018 • יד 4,"49,900 ₪","חסכוני,גלגלי מגנזיום",Renault Megane Grand Coupe,Intense OT diesel 1.5 (110 hp),"Economical, magnesium wheels"
19727,ב.מ.וו סדרה 1,118I S אוט' 5 דל' 1.5 (136 כ''ס),2018 • יד 2,לא צוין מחיר,"בקרת שמירת מרחק,בהזדמנות",BMW 1 Series,118I S Ot' 5 Del. 1.5 (136 HP),"Distance control, on occasion"
19728,ניסאן ג'וק / Juke,"Acenta אוט' 1.6 (117 כ""ס)",2016 • יד 3,"37,990 ₪","גלגלי מגנזיום,טסט בתוקף",Nissan Juke / Juke,Acenta auto 1.6 (117 hp),"Magnesium wheels, fully tested"
